In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import numpy as np
from household_package.data import call_data_url
from household_package.clean import clean_data
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from household_package.model import get_xy
from household_package.clean import filter_data
from household_package.registry import load_model
import pandas as pd
from household_package.registry import save_model
import pickle
from sklearn.model_selection import cross_validate
from household_package.preprocessor import preprocessing
from google.cloud import storage
from household_package.clean import clean_data_without

In [4]:
BUCKET_NAME="01_household_energy"
LOCAL_REGISTRY_PATH = "../model_h5/loaded/"


In [5]:
df_raw = call_data_url()

In [3]:
user_input = user_input = {
  "TYPEHUQ": 1,
  "NHSLDMEM": 1,
  "state_name": "Alabama",
  "REGIONC": "SOUTH",
  "BA_climate": "Hot-Humid",
  "SQFTEST": 240,
  "STORIES": 1,
  "YEARMADERANGE": 1,
  "NCOMBATH": 0,
  "NHAFBATH": 0,
  "TOTROOMS": 1,
  "WALLTYPE": 1,
  "ROOFTYPE": 1,
  "WINDOWS": 1,
  "SWIMPOOL": 1,
  "SOLAR": 1,
  "SMARTMETER": 1,
  "TELLWORK": 1,
  "DESKTOP": 0,
  "NUMLAPTOP": 0,
  "TVCOLOR": 0,
  "DISHWASH": 1,
  "MICRO": 0,
  "NUMFRIG": 0,
  "CWASHER": 1,
  "DRYER": 1,
  "LGTIN1TO4": 0,
  "LGTIN4TO8": 0,
  "LGTINMORE8": 0,
  "AIRCOND": 1,
  "EQUIPM": 3,
  "HEATHOME": 1,
  "NUMPORTEL": 0
}

In [121]:
def make_X_new(user_input):
    """
    This functions takes a dictionary coming from user inputs
    and makes a one row of X for model prediciton.
    """
    ## get all the passed arguments
    #user_inputs = locals().copy()
    X_new = pd.DataFrame({k:[v] for k,v in user_input.items()})
    return X_new


In [42]:
X_new = make_X_new(user_input)

In [28]:
#X_row_new = pd.DataFrame(pd.Series(user_input)).T

In [43]:
X_row_new = clean_data(X_new)

In [44]:
X_row_new

,SWIMPOOL,NUMPORTEL,SOLAR,SMARTMETER,TYPEHUQ,NHSLDMEM,state_name,REGIONC,BA_climate,SQFTEST,...,MICRO,NUMFRIG,CWASHER,DRYER,AIRCOND,EQUIPM,HEATHOME,TOTAL_BATH,TOTAL_COMP,TOTAL_LIGHT
0,1,0,1,1,Mobile,1,Alabama,SOUTH,Hot-Humid,240,...,0,0,1,1,1,Furnace,1,0,0,0


In [59]:
from google.cloud import storage
client = storage.Client()
blobs = list(client.get_bucket(BUCKET_NAME).list_blobs(prefix="model"))

In [60]:
blobs

[<Blob: 01_household_energy, models/baseline_20231129-122043.pkl, 1701256844126884>,
 <Blob: 01_household_energy, models/baseline_20231129-131320.pkl, 1701260000941981>,
 <Blob: 01_household_energy, models/baseline_20231130-102940.pkl, 1701337090569311>]

In [101]:
latest_blob = max(blobs, key=lambda x: x.updated)
#print(latest_blob.name)
latest_model_path_to_save = os.path.join(LOCAL_REGISTRY_PATH, latest_blob.name)
print(latest_model_path_to_save)
latest_blob.download_to_filename(latest_model_path_to_save)
# for sklearn (baseline)
with open(latest_model_path_to_save , 'rb') as f:
    latest_model = pickle.load(f)

../model_h5/loaded/models/baseline_20231130-102940.pkl


In [70]:
latest_model.predict(X_row_new)

array([11040.])

In [31]:

#percentile_min = df_raw['KWH'].quantile(0.005)
#percentile_max = df_raw['KWH'].quantile(0.995)

# Filter the DataFrame based on the specified percentiles
#filtered_df = df_raw[(df_raw['KWH'] >= percentile_min) & (df_raw['KWH'] <= percentile_max)]
#filtered_df

In [83]:
df = clean_data(df_raw)

In [177]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

def filter_data(df,min_perc=0.005, max_perc=0.995):
    df = df.copy()

    #Get rid of outliers#
    percentile_min = df['KWH'].quantile(min_perc)
    percentile_max = df['KWH'].quantile(max_perc)

    # Filter the DataFrame based on the specified percentiles
    df = df[(df['KWH'] >= percentile_min) & (df['KWH'] <= percentile_max)]
    return df


def clean_data_2(df,min_perc=0.005, max_perc=0.995):
    """This set cleans the data to have it in the correct format before preprocessing"""

    #Maping features
    TYPEHUQ_map = {1: "Mobile", 2: "Single_detached",3: "Single_attached",
                   4: "Appartment_small" ,5: "Appartment_big"}
    
    YEARMADERANGE_map = {1: "Before_50", 2:"50-59", 3:"60-69",
                         4: "70-79", 5: "80-89", 6:"90-99", 7:"00-09",
                         8:"2010-15", 9:"2016-20" }
    EQUIPM_map = {3:"Furnace", 2: "Steam", 4: "Central_heat", 13: "Ductless_heat",
                5: "Electric_units", 7: "Room_heater", 8 : "Wood",
                10:"Electric_heater", 99:"Other", -2:"Other"}

    dics_transform = [TYPEHUQ_map, YEARMADERANGE_map, EQUIPM_map]
    columns_to_transform = ["TYPEHUQ", "YEARMADERANGE", "EQUIPM"]

    for x , y in zip(dics_transform,columns_to_transform):
        df[y] = df[y].map(x)

    #Combine new features
    df["TOTAL_BATH"] = df["NCOMBATH"] + df["NHAFBATH"]
    df.drop( ["NCOMBATH", "NHAFBATH"] ,axis=1, inplace = True)

    df["TOTAL_COMP"] = df["DESKTOP"] + df["NUMLAPTOP"]
    df.drop( ["DESKTOP", "NUMLAPTOP"] ,axis=1, inplace = True)

    df["TOTAL_LIGHT"] = df["LGTIN1TO4"] + df["LGTIN4TO8"] + df["LGTINMORE8"]
    df.drop( ["LGTIN1TO4", "LGTIN4TO8", "LGTINMORE8"] ,axis=1, inplace = True)

    df["STORIES"] = df["STORIES"].replace(-2,1)
    df["STORIES"] = df["STORIES"].replace(5,2)

    #Impute features
    features_imputer2 = ['SWIMPOOL', 'NUMPORTEL', 'SOLAR']
    features_imputer4 = ['SMARTMETER']
    #to_ohe_encode = ['REGIONC', 'state_name','BA_climate','TYPEHUQ','YEARMADERANGE','WALLTYPE','ROOFTYPE','WINDOWS','EQUIPM']

    imputer_2 = SimpleImputer(strategy='constant', missing_values=-2, fill_value=0)
    imputer_4 = SimpleImputer(strategy='constant', missing_values=-4, fill_value=0)
    #ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

    preprocessor = ColumnTransformer(transformers=[('imputer_2',imputer_2, features_imputer2),
                                                ('imputer_4',imputer_4, features_imputer4)],
                                                #('ohe', ohe, to_ohe_encode)],
                                                    remainder = "passthrough")

    preprocessor.fit(df)

    cols = [x.split("__")[1] for x in preprocessor.get_feature_names_out()]

    df_clean = pd.DataFrame(preprocessor.transform(df),
                       columns = cols)

    return df_clean


In [178]:
df = call_data_url()

In [179]:
df = df.drop(["WALLTYPE", "ROOFTYPE"],axis=1)

In [180]:
df = clean_data_2(df)

In [181]:
df.columns

Index(['SWIMPOOL', 'NUMPORTEL', 'SOLAR', 'SMARTMETER', 'REGIONC', 'state_name',
       'BA_climate', 'TYPEHUQ', 'STORIES', 'YEARMADERANGE', 'TOTROOMS',
       'WINDOWS', 'NUMFRIG', 'MICRO', 'DISHWASH', 'CWASHER', 'DRYER',
       'TVCOLOR', 'TELLWORK', 'HEATHOME', 'EQUIPM', 'AIRCOND', 'NHSLDMEM',
       'SQFTEST', 'KWH', 'TOTAL_BATH', 'TOTAL_COMP', 'TOTAL_LIGHT'],
      dtype='object')

In [182]:
X , y = get_xy(df)

X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size=0.3)

In [183]:
to_ohe_encode = ['REGIONC', 'state_name','BA_climate','TYPEHUQ', "EQUIPM",
                     'YEARMADERANGE', 'WINDOWS']

to_scale = ["NUMPORTEL", "STORIES","SQFTEST",
            "TOTROOMS", "NUMFRIG", "MICRO", "TVCOLOR","NHSLDMEM",
            "TOTAL_BATH", "TOTAL_COMP", "TOTAL_LIGHT" ]

min_max = MinMaxScaler()
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')


preprocessor = ColumnTransformer(transformers=[
                                            ('min_max', min_max, to_scale),
                                            ('ohe', ohe, to_ohe_encode)],
                                            remainder = "passthrough")

model = LinearRegression()

pipe = Pipeline([('prep', preprocessor), ('model', model)])

### Score - two features

In [184]:
y_train_p = np.log(y_train.astype("float"))
y_test_p = np.log(y_test.astype("float"))

In [185]:
cross_validate(pipe, X_train, y_train, cv =5)["test_score"].mean()

0.5064290079231559

In [186]:
cross_validate(pipe, X_train, y_train_p, cv =5)["test_score"].mean()

0.5390166210563248

In [187]:
pipe.fit(X_train, y_train_p)

Pipeline(steps=[('prep',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('min_max', MinMaxScaler(),
                                                  ['NUMPORTEL', 'STORIES',
                                                   'SQFTEST', 'TOTROOMS',
                                                   'NUMFRIG', 'MICRO',
                                                   'TVCOLOR', 'NHSLDMEM',
                                                   'TOTAL_BATH', 'TOTAL_COMP',
                                                   'TOTAL_LIGHT']),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['REGIONC', 'state_name',
                                                   'BA_climate', 'TYPEHUQ',
                                                   'EQUIPM', 'YEARMADERANGE',
                                                   'WINDOWS'])])),
                ('model', LinearRegression())])

In [188]:
pipe.score(X_test, y_test_p)

0.5248544499483565

In [215]:
preprocessor.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('min_max', MinMaxScaler(),
                                 ['NUMPORTEL', 'STORIES', 'SQFTEST', 'TOTROOMS',
                                  'NUMFRIG', 'MICRO', 'TVCOLOR', 'NHSLDMEM',
                                  'TOTAL_BATH', 'TOTAL_COMP', 'TOTAL_LIGHT']),
                                ('ohe',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['REGIONC', 'state_name', 'BA_climate',
                                  'TYPEHUQ', 'EQUIPM', 'YEARMADERANGE',
                                  'WINDOWS'])])

In [216]:
X_trans = preprocessing.transform(X_train)

AttributeError: 'function' object has no attribute 'transform'

In [198]:
preprocessor.transform(X_test)

array([[0.0, 0.3333333333333333, 0.10348360655737704, ..., 0, 1, 1],
       [0.0, 0.0, 0.07581967213114754, ..., 0, 1, 1],
       [0.0, 0.0, 0.13934426229508196, ..., 1, 0, 1],
       ...,
       [0.0, 0.0, 0.05737704918032786, ..., 0, 1, 1],
       [0.0, 0.3333333333333333, 0.11885245901639344, ..., 0, 0, 0],
       [0.0, 0.3333333333333333, 0.2827868852459017, ..., 1, 1, 1]],
      dtype=object)

In [199]:
model = LinearRegression()

In [204]:
pd.DataFrame(y_train_p)

,KWH
12406,9.299265
2216,9.169833
12653,9.929013
7637,9.655036
7773,8.503828
...,...
11333,9.552401
11213,8.983439
12809,9.314577
427,9.867064


In [210]:
y_train_p

12406    9.299265
2216     9.169833
12653    9.929013
7637     9.655036
7773     8.503828
           ...   
11333    9.552401
11213    8.983439
12809    9.314577
427      9.867064
15594    9.512615
Name: KWH, Length: 12947, dtype: float64

In [209]:
model.fit(X_trans, pd.DataFrame(y_train_p).reshape(-1,1))

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [192]:
make_X_new(X_test.iloc[0,:])

,SWIMPOOL,NUMPORTEL,SOLAR,SMARTMETER,REGIONC,state_name,BA_climate,TYPEHUQ,STORIES,YEARMADERANGE,...,TVCOLOR,TELLWORK,HEATHOME,EQUIPM,AIRCOND,NHSLDMEM,SQFTEST,TOTAL_BATH,TOTAL_COMP,TOTAL_LIGHT
0,0,0,0,0,NORTHEAST,New York,Cold,Single_detached,2,Before_50,...,1,0,1,Furnace,1,2,1250,2,3,4


In [193]:
pipe.predict(make_X_new(X_test.iloc[0,:]))

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [158]:
make_X_new(X_test.iloc[0,:])

,SWIMPOOL,NUMPORTEL,SOLAR,SMARTMETER,REGIONC,state_name,BA_climate,TYPEHUQ,STORIES,YEARMADERANGE,...,TVCOLOR,TELLWORK,HEATHOME,EQUIPM,AIRCOND,NHSLDMEM,SQFTEST,TOTAL_BATH,TOTAL_COMP,TOTAL_LIGHT
0,0,0,0,1,SOUTH,Florida,Hot-Humid,Single_detached,1,2016-20,...,4,0,0,Other,1,7,2200,2,1,6


In [159]:
pipe.predict(make_X_new(X_test.iloc[0,:]))

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

### Score - all features

In [111]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

def filter_data(df,min_perc=0.005, max_perc=0.995):
    df = df.copy()

    #Get rid of outliers#
    percentile_min = df['KWH'].quantile(min_perc)
    percentile_max = df['KWH'].quantile(max_perc)

    # Filter the DataFrame based on the specified percentiles
    df = df[(df['KWH'] >= percentile_min) & (df['KWH'] <= percentile_max)]
    return df


def clean_data(df,min_perc=0.005, max_perc=0.995):
    """This set cleans the data to have it in the correct format before preprocessing"""

    #Maping features
    TYPEHUQ_map = {1: "Mobile", 2: "Single_detached",3: "Single_attached",
                   4: "Appartment_small" ,5: "Appartment_big"}
    YEARMADERANGE_map = {1: "Before_50", 2:"50-59", 3:"60-69",
                         4: "70-79", 5: "80-89", 6:"90-99", 7:"00-09",
                         8:"2010-15", 9:"2016-20" }
    WALLTYPE_map = {1:"Brick",2:"Wood",3:"Siding",4:"Stucco",5:"Shingle",
                    6:"Stone",7:"Concrete",99:"Other"}
    ROOFTYPE_map = {1 : "Ceramic", 2 : "Wood", 3 : "Metal", 4: "Slate",
                    5: "Shingle", 6: "Concrete", 99: "Other", -2:"Other"}
    EQUIPM_map = {3:"Furnace", 2: "Steam", 4: "Central_heat", 13: "Ductless_heat",
                5: "Electric_units", 7: "Room_heater", 8 : "Wood",
                10:"Electric_heater", 99:"Other", -2:"Other"}

    dics_transform = [TYPEHUQ_map, YEARMADERANGE_map, WALLTYPE_map, ROOFTYPE_map, EQUIPM_map]
    columns_to_transform = ["TYPEHUQ", "YEARMADERANGE", "WALLTYPE", "ROOFTYPE", "EQUIPM"]

    for x , y in zip(dics_transform,columns_to_transform):
        df[y] = df[y].map(x)

    #Combine new features
    df["TOTAL_BATH"] = df["NCOMBATH"] + df["NHAFBATH"]
    df.drop( ["NCOMBATH", "NHAFBATH"] ,axis=1, inplace = True)

    df["TOTAL_COMP"] = df["DESKTOP"] + df["NUMLAPTOP"]
    df.drop( ["DESKTOP", "NUMLAPTOP"] ,axis=1, inplace = True)

    df["TOTAL_LIGHT"] = df["LGTIN1TO4"] + df["LGTIN4TO8"] + df["LGTINMORE8"]
    df.drop( ["LGTIN1TO4", "LGTIN4TO8", "LGTINMORE8"] ,axis=1, inplace = True)

    df["STORIES"] = df["STORIES"].replace(-2,1)
    df["STORIES"] = df["STORIES"].replace(5,2)

    #Impute features
    features_imputer2 = ['SWIMPOOL', 'NUMPORTEL', 'SOLAR']
    features_imputer4 = ['SMARTMETER']
    #to_ohe_encode = ['REGIONC', 'state_name','BA_climate','TYPEHUQ','YEARMADERANGE','WALLTYPE','ROOFTYPE','WINDOWS','EQUIPM']

    imputer_2 = SimpleImputer(strategy='constant', missing_values=-2, fill_value=0)
    imputer_4 = SimpleImputer(strategy='constant', missing_values=-4, fill_value=0)
    #ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

    preprocessor = ColumnTransformer(transformers=[('imputer_2',imputer_2, features_imputer2),
                                                ('imputer_4',imputer_4, features_imputer4)],
                                                #('ohe', ohe, to_ohe_encode)],
                                                    remainder = "passthrough")

    preprocessor.fit(df)

    cols = [x.split("__")[1] for x in preprocessor.get_feature_names_out()]

    df_clean = pd.DataFrame(preprocessor.transform(df),
                       columns = cols)

    return df_clean

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler


to_ohe_encode = ['REGIONC', 'state_name','BA_climate','TYPEHUQ',
                     'YEARMADERANGE','WALLTYPE','ROOFTYPE','WINDOWS','EQUIPM']
to_scale = ["NUMPORTEL", "STORIES","SQFTEST",
            "TOTROOMS", "NUMFRIG", "MICRO", "TVCOLOR","NHSLDMEM",
            "TOTAL_BATH", "TOTAL_COMP", "TOTAL_LIGHT" ]

min_max = MinMaxScaler()
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')


preprocessor = ColumnTransformer(transformers=[
                                            ('min_max', min_max, to_scale),
                                            ('ohe', ohe, to_ohe_encode)],
                                            remainder = "passthrough")

model = LinearRegression()

pipe_2 = Pipeline([('prep', preprocessor), ('model', model)])

In [112]:
df = call_data_url()

In [113]:
df = clean_data(df)

In [114]:
df.columns

Index(['SWIMPOOL', 'NUMPORTEL', 'SOLAR', 'SMARTMETER', 'REGIONC', 'state_name',
       'BA_climate', 'TYPEHUQ', 'STORIES', 'YEARMADERANGE', 'TOTROOMS',
       'WALLTYPE', 'ROOFTYPE', 'WINDOWS', 'NUMFRIG', 'MICRO', 'DISHWASH',
       'CWASHER', 'DRYER', 'TVCOLOR', 'TELLWORK', 'HEATHOME', 'EQUIPM',
       'AIRCOND', 'NHSLDMEM', 'SQFTEST', 'KWH', 'TOTAL_BATH', 'TOTAL_COMP',
       'TOTAL_LIGHT'],
      dtype='object')

In [115]:
X , y = get_xy(df)
X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size=0.3)

y_train_p = np.log(y_train.astype("float"))
y_test_p = np.log(y_test.astype("float"))

In [116]:
cross_validate(pipe_2, X_train, y_train, cv =5)["test_score"].mean()

0.48715139693386833

In [117]:
cross_validate(pipe_2, X_train, y_train_p, cv =5)["test_score"].mean()

0.5400254484470768

In [118]:
pipe_2.fit(X_train, y_train_p)

Pipeline(steps=[('prep',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('min_max', MinMaxScaler(),
                                                  ['NUMPORTEL', 'STORIES',
                                                   'SQFTEST', 'TOTROOMS',
                                                   'NUMFRIG', 'MICRO',
                                                   'TVCOLOR', 'NHSLDMEM',
                                                   'TOTAL_BATH', 'TOTAL_COMP',
                                                   'TOTAL_LIGHT']),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['REGIONC', 'state_name',
                                                   'BA_climate', 'TYPEHUQ',
                                                   'YEARMADERANGE', 'WALLTYPE',
                                                   'ROOFTYPE', 'WINDOWS',
                                                   'EQUIPM'])])),
                ('model', LinearRegression())])

In [119]:
pipe_2.score(X_test, y_test_p)

0.5259557463577558

### Random Regressor

In [312]:
df = call_data_url()

In [314]:
df = filter_data(df)
df = clean_data(df)

In [317]:
X , y = get_xy(df)
X_train , X_test, y_train , y_test = train_test_split(X, y , test_size=0.3)

In [43]:
#Needed for X predict
pd.DataFrame(X.iloc[0,:]).T

,SWIMPOOL,NUMPORTEL,SOLAR,SMARTMETER,REGIONC,state_name,BA_climate,TYPEHUQ,STORIES,YEARMADERANGE,...,TVCOLOR,TELLWORK,HEATHOME,EQUIPM,AIRCOND,NHSLDMEM,SQFTEST,TOTAL_BATH,TOTAL_COMP,TOTAL_LIGHT
0,0,0,0,0,WEST,New Mexico,Mixed-Dry,Single_detached,1,70-79,...,3,0,1,Furnace,1,2,2100,3,1,17


In [318]:
from sklearn.ensemble import RandomForestRegressor

to_ohe_encode = ['REGIONC', 'state_name','BA_climate','TYPEHUQ',
                     'YEARMADERANGE','WALLTYPE','ROOFTYPE','WINDOWS','EQUIPM']

to_scale = ["NUMPORTEL", "STORIES","SQFTEST",
            "TOTROOMS", "NUMFRIG", "MICRO", "TVCOLOR","NHSLDMEM",
            "TOTAL_BATH", "TOTAL_COMP", "TOTAL_LIGHT" ]

min_max = MinMaxScaler()
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')


preprocessor = ColumnTransformer(transformers=[
                                            ('min_max', min_max, to_scale),
                                            ('ohe', ohe, to_ohe_encode)],
                                            remainder = "passthrough")

model = RandomForestRegressor()

pipe_2 = Pipeline([('prep', preprocessor), ('model', model)])

In [13]:
pipe_2.fit(X_train,y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('min_max', MinMaxScaler(),
                                                  ['NUMPORTEL', 'STORIES',
                                                   'SQFTEST', 'TOTROOMS',
                                                   'NUMFRIG', 'MICRO',
                                                   'TVCOLOR', 'NHSLDMEM',
                                                   'TOTAL_BATH', 'TOTAL_COMP',
                                                   'TOTAL_LIGHT']),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['REGIONC', 'state_name',
                                                   'BA_climate', 'TYPEHUQ',
                                                   'YEARMADERANGE', 'WALLTYPE',
                                                   'ROOFTYPE', 'WINDOWS',
                                                   'EQUIPM'])])),
                ('model', RandomForestRegressor())])

In [15]:
pipe_2.score(X_test,y_test)

0.46195432428379035

In [320]:
preprocessor.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('min_max', MinMaxScaler(),
                                 ['NUMPORTEL', 'STORIES', 'SQFTEST', 'TOTROOMS',
                                  'NUMFRIG', 'MICRO', 'TVCOLOR', 'NHSLDMEM',
                                  'TOTAL_BATH', 'TOTAL_COMP', 'TOTAL_LIGHT']),
                                ('ohe',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['REGIONC', 'state_name', 'BA_climate',
                                  'TYPEHUQ', 'YEARMADERANGE', 'WALLTYPE',
                                  'ROOFTYPE', 'WINDOWS', 'EQUIPM'])])

In [321]:
X_train_prep = preprocessor.transform(X_train)

In [322]:
model = RandomForestRegressor()

In [323]:
model.fit(X_train_prep, y_train)

RandomForestRegressor()

In [331]:
pd.set_option('display.max_rows', None)

rfg_df = pd.DataFrame(model.feature_importances_, index=preprocessor.get_feature_names_out())

rfg_df.reset_index().sort_values(by="index")

,index,0
5,min_max__MICRO,0.005087
7,min_max__NHSLDMEM,0.057803
4,min_max__NUMFRIG,0.030783
0,min_max__NUMPORTEL,0.018243
2,min_max__SQFTEST,0.188624
1,min_max__STORIES,0.008564
8,min_max__TOTAL_BATH,0.016074
9,min_max__TOTAL_COMP,0.024478
10,min_max__TOTAL_LIGHT,0.055183
3,min_max__TOTROOMS,0.046265


### Testing model with all values

In [266]:
df = call_data_url()

In [267]:
df = filter_data(df)

In [268]:
df = clean_data(df)

In [269]:
X , y = get_xy(df)

In [270]:
X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size=0.3)
y_train_p = np.log(y_train.astype("float"))
y_test_p = np.log(y_test.astype("float"))

In [271]:
prep = preprocessing(X_train)

In [272]:
X_trans = prep.transform(X_train)

In [273]:
model = LinearRegression()

In [274]:
model.fit(X_trans, y_train)

LinearRegression()

In [275]:
cross_validate(model , X_trans, y_train, cv=5 )["test_score"].mean()

0.49268817807784976

In [276]:
cross_validate(model , X_trans, y_train_p, cv=5 )["test_score"].mean()

0.5366381288240587

In [277]:
X_test_trans = prep.transform(X_test)

In [293]:
pd.DataFrame(X_test_trans[0]).T

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.0,0.0,0.077053,0.285714,0.111111,0.333333,0.071429,0.5,0.2,0.090909,...,0.0,0,0,0,1,1,1,1,1,1


In [294]:
model.predict(pd.DataFrame(X_test_trans[0]).T)

array([9544.])

### Without Roof and Wall

In [295]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

def clean_data_2(df,min_perc=0.005, max_perc=0.995):
    """This set cleans the data to have it in the correct format before preprocessing"""

    #Maping features
    TYPEHUQ_map = {1: "Mobile", 2: "Single_detached",3: "Single_attached",
                   4: "Appartment_small" ,5: "Appartment_big"}
    
    YEARMADERANGE_map = {1: "Before_50", 2:"50-59", 3:"60-69",
                         4: "70-79", 5: "80-89", 6:"90-99", 7:"00-09",
                         8:"2010-15", 9:"2016-20" }
    EQUIPM_map = {3:"Furnace", 2: "Steam", 4: "Central_heat", 13: "Ductless_heat",
                5: "Electric_units", 7: "Room_heater", 8 : "Wood",
                10:"Electric_heater", 99:"Other", -2:"Other"}

    dics_transform = [TYPEHUQ_map, YEARMADERANGE_map, EQUIPM_map]
    columns_to_transform = ["TYPEHUQ", "YEARMADERANGE", "EQUIPM"]

    for x , y in zip(dics_transform,columns_to_transform):
        df[y] = df[y].map(x)

    #Combine new features
    df["TOTAL_BATH"] = df["NCOMBATH"] + df["NHAFBATH"]
    df.drop( ["NCOMBATH", "NHAFBATH"] ,axis=1, inplace = True)

    df["TOTAL_COMP"] = df["DESKTOP"] + df["NUMLAPTOP"]
    df.drop( ["DESKTOP", "NUMLAPTOP"] ,axis=1, inplace = True)

    df["TOTAL_LIGHT"] = df["LGTIN1TO4"] + df["LGTIN4TO8"] + df["LGTINMORE8"]
    df.drop( ["LGTIN1TO4", "LGTIN4TO8", "LGTINMORE8"] ,axis=1, inplace = True)

    df["STORIES"] = df["STORIES"].replace(-2,1)
    df["STORIES"] = df["STORIES"].replace(5,2)

    #Impute features
    features_imputer2 = ['SWIMPOOL', 'NUMPORTEL', 'SOLAR']
    features_imputer4 = ['SMARTMETER']
    #to_ohe_encode = ['REGIONC', 'state_name','BA_climate','TYPEHUQ','YEARMADERANGE','WALLTYPE','ROOFTYPE','WINDOWS','EQUIPM']

    imputer_2 = SimpleImputer(strategy='constant', missing_values=-2, fill_value=0)
    imputer_4 = SimpleImputer(strategy='constant', missing_values=-4, fill_value=0)
    #ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

    preprocessor = ColumnTransformer(transformers=[('imputer_2',imputer_2, features_imputer2),
                                                ('imputer_4',imputer_4, features_imputer4)],
                                                #('ohe', ohe, to_ohe_encode)],
                                                    remainder = "passthrough")

    preprocessor.fit(df)

    cols = [x.split("__")[1] for x in preprocessor.get_feature_names_out()]

    df_clean = pd.DataFrame(preprocessor.transform(df),
                       columns = cols)

    return df_clean


In [348]:
df_new = call_data_url()

In [349]:
df = filter_data(df_new)
df = clean_data_2(df_new)

In [350]:
df = df.drop(["WALLTYPE", "ROOFTYPE"], axis = 1)

In [351]:
df.columns

Index(['SWIMPOOL', 'NUMPORTEL', 'SOLAR', 'SMARTMETER', 'REGIONC', 'state_name',
       'BA_climate', 'TYPEHUQ', 'STORIES', 'YEARMADERANGE', 'TOTROOMS',
       'WINDOWS', 'NUMFRIG', 'MICRO', 'DISHWASH', 'CWASHER', 'DRYER',
       'TVCOLOR', 'TELLWORK', 'HEATHOME', 'EQUIPM', 'AIRCOND', 'NHSLDMEM',
       'SQFTEST', 'KWH', 'TOTAL_BATH', 'TOTAL_COMP', 'TOTAL_LIGHT'],
      dtype='object')

In [364]:
df_new = call_data_url()
df = df_new.drop(["WALLTYPE", "ROOFTYPE"], axis = 1)
df = filter_data(df)
df = clean_data_2(df)
X , y = get_xy(df)
X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size=0.3)
y_train_p = np.log(y_train.astype("float"))
y_test_p = np.log(y_test.astype("float"))

In [365]:
from sklearn.ensemble import RandomForestRegressor

to_ohe_encode = ['REGIONC', 'state_name','BA_climate','TYPEHUQ',
                     'YEARMADERANGE','WINDOWS','EQUIPM']

to_scale = ["NUMPORTEL", "STORIES","SQFTEST",
            "TOTROOMS", "NUMFRIG", "MICRO", "TVCOLOR","NHSLDMEM",
            "TOTAL_BATH", "TOTAL_COMP", "TOTAL_LIGHT" ]

min_max = MinMaxScaler()
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')


prep2 = ColumnTransformer(transformers=[
                                            ('min_max', min_max, to_scale),
                                            ('ohe', ohe, to_ohe_encode)],
                                            remainder = "passthrough")

pipe = Pipeline([("prep", prep2 ),("model", LinearRegression())])

In [366]:
pipe.fit(X_train, y_train_p)

Pipeline(steps=[('prep',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('min_max', MinMaxScaler(),
                                                  ['NUMPORTEL', 'STORIES',
                                                   'SQFTEST', 'TOTROOMS',
                                                   'NUMFRIG', 'MICRO',
                                                   'TVCOLOR', 'NHSLDMEM',
                                                   'TOTAL_BATH', 'TOTAL_COMP',
                                                   'TOTAL_LIGHT']),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['REGIONC', 'state_name',
                                                   'BA_climate', 'TYPEHUQ',
                                                   'YEARMADERANGE', 'WINDOWS',
                                                   'EQUIPM'])])),
                ('model', LinearRegression())])

In [367]:
cross_validate(pipe, X_train, y_train_p, cv = 5)["test_score"].mean()

0.5392127310243101

## Elastic Net

In [22]:
df_new = call_data_url()
df = df_new.drop(["WALLTYPE", "ROOFTYPE"], axis = 1)
df = filter_data(df)
df = clean_data_without(df)
X , y = get_xy(df)
X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size=0.3)
y_train_p = np.log(y_train.astype("float"))
y_test_p = np.log(y_test.astype("float"))

In [31]:
X_train

,SWIMPOOL,NUMPORTEL,SOLAR,SMARTMETER,REGIONC,state_name,BA_climate,TYPEHUQ,STORIES,YEARMADERANGE,...,TVCOLOR,TELLWORK,HEATHOME,EQUIPM,AIRCOND,NHSLDMEM,SQFTEST,TOTAL_BATH,TOTAL_COMP,TOTAL_LIGHT
2004,0,0,0,0,SOUTH,Georgia,Mixed-Humid,Single_detached,2,2016-20,...,4,0,1,Furnace,1,2,2980,3,1,22
1580,0,0,0,0,SOUTH,Kentucky,Mixed-Humid,Single_detached,1,Before_50,...,2,0,1,Furnace,1,5,1180,1,0,12
2558,0,0,0,0,SOUTH,North Carolina,Mixed-Humid,Single_detached,1,60-69,...,1,0,1,Furnace,1,1,1650,2,0,3
17282,0,0,0,0,SOUTH,Virginia,Mixed-Humid,Single_detached,1,50-59,...,1,0,1,Furnace,1,1,1250,1,0,2
12635,1,1,0,0,WEST,California,Hot-Dry,Single_detached,2,Before_50,...,3,1,1,Furnace,1,3,3070,4,2,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7481,0,0,0,0,WEST,Nevada,Hot-Dry,Appartment_big,1,90-99,...,1,0,1,Furnace,1,2,620,1,0,8
4068,0,0,0,0,NORTHEAST,Connecticut,Cold,Single_detached,1,70-79,...,2,0,1,Electric_units,1,2,1700,2,2,15
13600,0,2,0,1,WEST,Oregon,Cold,Single_detached,1,Before_50,...,1,0,1,Electric_heater,0,1,1300,2,1,5
7729,0,0,0,1,MIDWEST,Ohio,Cold,Mobile,1,00-09,...,3,0,1,Furnace,1,4,1280,2,3,17


In [32]:
from sklearn.linear_model import ElasticNet

to_ohe_encode = ['REGIONC', 'state_name','BA_climate','TYPEHUQ',
                     'YEARMADERANGE','WINDOWS','EQUIPM']

to_scale = ["NUMPORTEL", "STORIES","SQFTEST",
            "TOTROOMS", "NUMFRIG", "MICRO", "TVCOLOR","NHSLDMEM",
            "TOTAL_BATH", "TOTAL_COMP", "TOTAL_LIGHT" ]

min_max = MinMaxScaler()
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')


prep2 = ColumnTransformer(transformers=[
                                            ('min_max', min_max, to_scale),
                                            ('ohe', ohe, to_ohe_encode)],
                                            remainder = "passthrough")

pipe = Pipeline([("prep", prep2 ),("model", ElasticNet())])

In [28]:
from sklearn.model_selection import GridSearchCV

In [33]:
params = {'model__alpha':[0.001, 0.01, 0.1, 1, 10], 
          'model__l1_ratio':[0.1,0.2, 0.5,0.8]}

In [34]:
search = GridSearchCV(pipe, param_grid=params)

In [35]:
search.fit(X_train, y_train_p)

GridSearchCV(estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('min_max',
                                                                         MinMaxScaler(),
                                                                         ['NUMPORTEL',
                                                                          'STORIES',
                                                                          'SQFTEST',
                                                                          'TOTROOMS',
                                                                          'NUMFRIG',
                                                                          'MICRO',
                                                                          'TVCOLOR',
                                                                          'NHSLDMEM',
                                                                          'TOTAL_BATH',
                                                                          'TOTAL_COMP',
                                                                          'TOTAL_LIGHT']),
                                                                        ('ohe',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         ['REGIONC',
                                                                          'state_name',
                                                                          'BA_climate',
                                                                          'TYPEHUQ',
                                                                          'YEARMADERANGE',
                                                                          'WINDOWS',
                                                                          'EQUIPM'])])),
                                       ('model', ElasticNet())]),
             param_grid={'model__alpha': [0.001, 0.01, 0.1, 1, 10],
                         'model__l1_ratio': [0.1, 0.2, 0.5, 0.8]})

In [36]:
search.best_score_

0.5422175834725463

In [30]:
cross_validate(pipe, X_train, y_train_p, cv =5 )

{'fit_time': array([0.12209702, 0.13957977, 0.14752269, 0.1435442 , 0.14555979]),
 'score_time': array([0.03126574, 0.02787924, 0.02887201, 0.04096174, 0.02188373]),
 'test_score': array([-1.75000429e-06, -2.31601231e-06, -1.36734691e-06, -4.78692418e-04,
        -5.80409686e-04])}

In [15]:
X_trans = prep2.transform(X_train)
X_trans_test = prep2.transform(X_test)

NotFittedError: This ColumnTransformer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
model = LinearRegression()
model.fit(X_trans, y_train)

In [309]:
cross_validate(model , X_trans, y_train, cv=5 )["test_score"].mean()

0.49769718217717623

In [310]:
cross_validate(model , X_trans, y_train_p, cv=5 )["test_score"].mean()

0.5302786418145256

In [339]:
from household_package.registry import load_model_locally_log
import os 

In [340]:
client = storage.Client()
blobs = list(client.get_bucket(BUCKET_NAME).list_blobs(prefix="model"))
print(blobs)
    #print(blobs)
latest_blob = max(blobs, key=lambda x: x.updated)
        #print(latest_blob.name)
latest_model_path_to_save = os.path.join(LOCAL_REGISTRY_PATH, latest_blob.name)
latest_blob.download_to_filename(latest_model_path_to_save)

with open(latest_model_path_to_save , 'rb') as f:
    latest_model = pickle.load(f)

[<Blob: 01_household_energy, models/baseline_20231129-122043.pkl, 1701256844126884>, <Blob: 01_household_energy, models/baseline_20231129-131320.pkl, 1701260000941981>, <Blob: 01_household_energy, models/baseline_20231130-102940.pkl, 1701337090569311>, <Blob: 01_household_energy, models/baseline_20231201-151102.pkl, 1701439862204187>]


In [341]:
latest_model

Pipeline(steps=[('prep',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('min_max', MinMaxScaler(),
                                                  ['NUMPORTEL', 'STORIES',
                                                   'SQFTEST', 'TOTROOMS',
                                                   'NUMFRIG', 'MICRO',
                                                   'TVCOLOR', 'NHSLDMEM',
                                                   'TOTAL_BATH', 'TOTAL_COMP',
                                                   'TOTAL_LIGHT']),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['REGIONC', 'state_name',
                                                   'BA_climate', 'TYPEHUQ',
                                                   'YEARMADERANGE', 'WINDOWS',
                                                   'EQUIPM'])])),
                ('model', LinearRegression())])

In [346]:
X , y = get_xy(df)
X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size=0.3)
y_train_p = np.log(y_train.astype("float"))
y_test_p = np.log(y_test.astype("float"))

In [347]:
latest_model.score(X_test, y_test_p)

0.5491828147668718

#### Linear Regression

In [84]:
df_new = call_data_url()
df = df_new.drop(["WALLTYPE", "ROOFTYPE", "TELLWORK","SOLAR", "SMARTMETER"], axis = 1)
df = filter_data(df)
df = clean_data_2(df)

X , y = get_xy(df)
X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size=0.3)
y_train_p = np.log(y_train.astype("float"))
y_test_p = np.log(y_test.astype("float"))

In [85]:
X_train.columns

Index(['SWIMPOOL', 'NUMPORTEL', 'REGIONC', 'state_name', 'BA_climate',
       'TYPEHUQ', 'STORIES', 'YEARMADERANGE', 'TOTROOMS', 'WINDOWS', 'NUMFRIG',
       'MICRO', 'DISHWASH', 'CWASHER', 'DRYER', 'TVCOLOR', 'HEATHOME',
       'EQUIPM', 'AIRCOND', 'NHSLDMEM', 'SQFTEST', 'TOTAL_BATH', 'TOTAL_COMP',
       'TOTAL_LIGHT'],
      dtype='object')

In [86]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

def clean_data_2(df,min_perc=0.005, max_perc=0.995):
    """This set cleans the data to have it in the correct format before preprocessing"""

    #Maping features
    TYPEHUQ_map = {1: "Mobile", 2: "Single_detached",3: "Single_attached",
                   4: "Appartment_small" ,5: "Appartment_big"}
    
    YEARMADERANGE_map = {1: "Before_50", 2:"50-59", 3:"60-69",
                         4: "70-79", 5: "80-89", 6:"90-99", 7:"00-09",
                         8:"2010-15", 9:"2016-20" }
    EQUIPM_map = {3:"Furnace", 2: "Steam", 4: "Central_heat", 13: "Ductless_heat",
                5: "Electric_units", 7: "Room_heater", 8 : "Wood",
                10:"Electric_heater", 99:"Other", -2:"Other"}

    dics_transform = [TYPEHUQ_map, YEARMADERANGE_map, EQUIPM_map]
    columns_to_transform = ["TYPEHUQ", "YEARMADERANGE", "EQUIPM"]

    for x , y in zip(dics_transform,columns_to_transform):
        df[y] = df[y].map(x)

    #Combine new features
    df["TOTAL_BATH"] = df["NCOMBATH"] + df["NHAFBATH"]
    df.drop( ["NCOMBATH", "NHAFBATH"] ,axis=1, inplace = True)

    df["TOTAL_COMP"] = df["DESKTOP"] + df["NUMLAPTOP"]
    df.drop( ["DESKTOP", "NUMLAPTOP"] ,axis=1, inplace = True)

    df["TOTAL_LIGHT"] = df["LGTIN1TO4"] + df["LGTIN4TO8"] + df["LGTINMORE8"]
    df.drop( ["LGTIN1TO4", "LGTIN4TO8", "LGTINMORE8"] ,axis=1, inplace = True)

    df["STORIES"] = df["STORIES"].replace(-2,1)
    df["STORIES"] = df["STORIES"].replace(5,2)

    #Impute features
    features_imputer2 = ['SWIMPOOL', 'NUMPORTEL']
    features_imputer4 = ['SMARTMETER']
    #to_ohe_encode = ['REGIONC', 'state_name','BA_climate','TYPEHUQ','YEARMADERANGE','WALLTYPE','ROOFTYPE','WINDOWS','EQUIPM']

    imputer_2 = SimpleImputer(strategy='constant', missing_values=-2, fill_value=0)
    imputer_4 = SimpleImputer(strategy='constant', missing_values=-4, fill_value=0)
    #ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

    preprocessor = ColumnTransformer(transformers=[('imputer_2',imputer_2, features_imputer2),
                                                #('imputer_4',imputer_4, features_imputer4)],
                                                #('ohe', ohe, to_ohe_encode)],
                                                  ], remainder = "passthrough")

    preprocessor.fit(df)

    cols = [x.split("__")[1] for x in preprocessor.get_feature_names_out()]

    df_clean = pd.DataFrame(preprocessor.transform(df),
                       columns = cols)

    return df_clean


In [78]:
df.columns

Index(['SWIMPOOL', 'NUMPORTEL', 'SMARTMETER', 'REGIONC', 'state_name',
       'BA_climate', 'TYPEHUQ', 'STORIES', 'YEARMADERANGE', 'TOTROOMS',
       'WINDOWS', 'NUMFRIG', 'MICRO', 'DISHWASH', 'CWASHER', 'DRYER',
       'TVCOLOR', 'HEATHOME', 'EQUIPM', 'AIRCOND', 'NHSLDMEM', 'SQFTEST',
       'KWH', 'TOTAL_BATH', 'TOTAL_COMP', 'TOTAL_LIGHT'],
      dtype='object')

In [87]:
from sklearn.linear_model import ElasticNet

to_ohe_encode = ['REGIONC', 'state_name','BA_climate','TYPEHUQ',
                     'YEARMADERANGE','WINDOWS','EQUIPM']

to_scale = ["NUMPORTEL", "STORIES","SQFTEST",
            "TOTROOMS", "NUMFRIG", "MICRO", "TVCOLOR","NHSLDMEM",
            "TOTAL_BATH", "TOTAL_COMP", "TOTAL_LIGHT" ]

min_max = MinMaxScaler()
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')


prep2 = ColumnTransformer(transformers=[
                                            ('min_max', min_max, to_scale),
                                            ('ohe', ohe, to_ohe_encode)],
                                            remainder = "passthrough")

pipe = Pipeline([("prep", prep2 ),("model", LinearRegression())])

In [80]:
#with Tellwork
cross_validate(pipe, X_train, y_train_p, cv=5)["test_score"].mean()

0.5401287742107884

In [81]:
#without Tellwork
cross_validate(pipe, X_train, y_train_p, cv=5)["test_score"].mean()

0.5401287742107884

In [88]:
cross_validate(pipe, X_train, y_train_p, cv=5)["test_score"].mean()

0.5380990169092557